# Download and Process the IMDB Dataset

In [14]:
# !pip install --quiet gdown

# # 1. Download the zipped IMDB dataset from Drive
# # this is the unsup part of https://ai.stanford.edu/~amaas/data/sentiment/

# !gdown "https://drive.google.com/uc?id=1PjJ5cop0pT6tcEw9-ZUstVMujx-o-QTB" -O imdb_dataset.zip

# # 2. Unzip the downloaded file
# !unzip -q imdb_dataset.zip -d imdb_data


In [1]:
import os
import re
import string
import random
from collections import defaultdict, Counter
import math
from math import log, exp
from nltk.corpus import stopwords
import numpy as np

In [2]:
def load_imdb_unsup_sentences(folder_path):
    """
    Loads text files from the IMDB 'unsup' (unsupervised) folder.
    Reads entire file content, replaces <br /> tags with spaced <nl> token,
    splits text by newline, and returns a list of raw lines.
    """
    all_sentences = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                content = file.read()
                content = content.replace("<br />", " <nl> ")
                for line in content.split('\n'):
                    line = line.strip()
                    if line:
                        all_sentences.append(line)
    return all_sentences

def remove_punctuation(text):
    """
    Removes punctuation from the text,
    but keeps <nl> tokens intact.
    """
    punctuation = ''.join(c for c in string.punctuation if c not in '<>')
    trans_table = str.maketrans('', '',punctuation)
    return text.translate(trans_table)

def build_vocabulary(sentences):
    """
    Builds vocabulary from sentences.
    Returns both vocabulary set and word counts.
    """
    word_counts = Counter()
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = remove_punctuation(sentence)
        tokens = sentence.split()
        word_counts.update(tokens)
    
    # Add special tokens
    vocab = set(word_counts.keys())
    vocab.update(['<s>', '</s>', '<UNK>'])
    return vocab

def tokenize(sentences, vocab, unknown="<UNK>"):
    """
    lower each sentence,
    Splits each sentence on whitespace, removes punctuation,
    and replaces tokens not in the vocabulary with unknowen token.
    Returns the list of tokenized sentences.
    """
    tokenized_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = remove_punctuation(sentence)
        tokens = sentence.split()
        tokenized_sentence = [token if token in vocab else unknown for token in tokens]
        tokenized_sentences.append(tokenized_sentence)
    return tokenized_sentences

In [3]:
imdb_folder = "imdb_dataset/unsup"
sentences = load_imdb_unsup_sentences(imdb_folder)

print(f"Number of raw sentences loaded: {len(sentences)}")
print(f"Example (first 2 sentences):\n{sentences[:2]}")

Number of raw sentences loaded: 50000
Example (first 2 sentences):
['I admit, the great majority of films released before say 1933 are just not for me. Of the dozen or so "major" silents I have viewed, one I loved (The Crowd), and two were very good (The Last Command and City Lights, that latter Chaplin circa 1931). <nl>  <nl> So I was apprehensive about this one, and humor is often difficult to appreciate (uh, enjoy) decades later. I did like the lead actors, but thought little of the film. <nl>  <nl> One intriguing sequence. Early on, the guys are supposed to get "de-loused" and for about three minutes, fully dressed, do some schtick. In the background, perhaps three dozen men pass by, all naked, white and black (WWI ?), and for most, their butts, part or full backside, are shown. Was this an early variation of beefcake courtesy of Howard Hughes?', 'Take a low budget, inexperienced actors doubling as production staff\x97 as well as limited facilities\x97and you can\'t expect much mor

In [4]:
assert len(sentences) == 50000, "Expected 50,000 sentences from the unsup folder."

In [5]:
random.seed(42)

def split_data(sentences, test_split=0.1):
  """
  Shuffle the sentences and split them into train and test sets.
  The first (1 - test_split) portion of the data is the training set.
  Returns the train and test sets.
  """
  random.shuffle(sentences)
  split_index = int(len(sentences) * (1 - test_split))
  train_sentences = sentences[:split_index]
  test_sentences = sentences[split_index:]
  return train_sentences, test_sentences


In [6]:
train_sentences, test_sentences = split_data(sentences)

print(f"Number of training sentences: {len(train_sentences)}")
print(f"Number of test sentences: {len(test_sentences)}")

Number of training sentences: 45000
Number of test sentences: 5000


In [7]:
assert len(train_sentences) == 45000, "Expected 45,000 sentences for training."
assert len(test_sentences) == 5000, "Expected 5,000 sentences for testing."


In [8]:
vocab = build_vocabulary(train_sentences)
tokenized_train = tokenize(train_sentences, vocab)
tokenized_test = tokenize(test_sentences, vocab)

print(f"Vocabulary size: {len(vocab)}")
print(f"Example tokens from first sentence: {tokenized_train[0][:10] if tokenized_train else 'No tokens loaded'} ...")


Vocabulary size: 161369
Example tokens from first sentence: ['having', 'first', 'seen', 'the', 'directors', '12min', 'take', 'on', 'poes', 'fall'] ...


In [9]:
# assert len(vocab) == 161292, "Expected a vocabulary size of 171,591." #skip for replication problems
assert len(tokenized_train) == 45000, "Expected tokenized sentences count to match raw sentences."

example = "I love Natural language processing, and i want to be a great engineer."
assert len(example) == 70, "Example sentence length (in characters) does not match the expected 70."

example_tokens = tokenize([example], vocab)[0]
assert len(example_tokens) == 13, "Token count for the example sentence does not match the expected 13."


In [10]:
def pad_sentence(tokens, n):
    """
    Pads a list of tokens with <s> at the start (n-1 times)
    and </s> at the end (once).
    For example, if n=3, you add 2 <s> tokens at the start.
    """
    padded = ['<s>'] * (n - 1) + tokens + ['</s>']
    return padded

def build_ngram_counts(tokenized_sentences, n, vocab):
    """
    Builds n-gram counts and (n-1)-gram counts from the given tokenized sentences.
    Each sentence is padded with <s> and </s>.

    Args:
        tokenized_sentences: list of lists, where each sub-list is a tokenized sentence.
        n: the order of the n-gram (e.g., 2 for bigrams, 3 for trigrams).
        vocab: set of known words. If provided, you can choose to handle out-of-vocab tokens.

    Returns:
        ngram_counts: Counter of n-grams (tuples of length n).
        context_counts: Counter of (n-1)-gram contexts.
    """
    ngram_counts = Counter()
    context_counts = Counter()

    for sentence in tokenized_sentences:
        padded_sentence = pad_sentence(sentence, n)
        for i in range(len(padded_sentence) - n + 1):
            ngram = tuple(padded_sentence[i:i + n])
            context = tuple(padded_sentence[i:i + n - 1])
            if all(token in vocab for token in ngram):
                ngram_counts[ngram] += 1
                context_counts[context] += 1

    return ngram_counts, context_counts

def laplace_probability(ngram, ngram_counts, context_counts, vocab_size, alpha=1.0):
    """
    Computes the probability of an n-gram using Laplace (add-alpha) smoothing.

    P(w_i | w_{i-(n-1)}, ..., w_{i-1}) =
        (count(ngram) + alpha) / (count(context) + alpha * vocab_size)

    Args:
        ngram: tuple of tokens representing the n-gram
        ngram_counts: Counter of n-grams
        context_counts: Counter of (n-1)-gram contexts
        vocab_size: size of the vocabulary
        alpha: smoothing parameter (1.0 = add-1 smoothing)

    Returns:
        Probability of the given n-gram.
    """
    context = ngram[:-1]
    ngram_count = ngram_counts.get(ngram, 0)
    context_count = context_counts.get(context, 0)
    prob = (ngram_count + alpha) / (context_count + alpha * vocab_size)
    return prob


In [11]:
# Unigram
n = 1
unigram_counts, unigram_context_counts = build_ngram_counts(tokenized_train, n=n, vocab=vocab)
print(f"Number of unigrams: {len(unigram_counts)}")
print(f"Number of unigram contexts: {len(unigram_context_counts)}")

# Bigram
n = 2
bigram_counts, bigram_context_counts = build_ngram_counts(tokenized_train, n=n, vocab=vocab)
print(f"Number of bigrams: {len(bigram_counts)}")
print(f"Number of bigram contexts: {len(bigram_context_counts)}")

# Trigram
n = 3
trigram_counts, trigram_context_counts = build_ngram_counts(tokenized_train, n=n, vocab=vocab)
print(f"Number of trigrams: {len(trigram_counts)}")
print(f"Number of trigram contexts: {len(trigram_context_counts)}")


Number of unigrams: 161367
Number of unigram contexts: 1
Number of bigrams: 2281351
Number of bigram contexts: 161367
Number of trigrams: 6113199
Number of trigram contexts: 2272846


In [12]:
from math import log, exp

def predict_next_token(
    context_tokens,
    ngram_counts,
    context_counts,
    vocab,
    n=2,
    alpha=1.0,
    top_k=5
):
    """
    Given a list of context tokens, predict the next token using the n-gram model.
    Returns the top_k predictions as (token, probability).
    """
    context = tuple(context_tokens[-(n-1):])
    candidates = []
    
    for token in vocab:
        ngram = context + (token,)
        prob = laplace_probability(ngram, ngram_counts, context_counts, len(vocab), alpha)
        candidates.append((token, prob))
    
    candidates.sort(key=lambda x: x[1], reverse=True)
    return candidates[:top_k]

def generate_text_with_limit(
    start_tokens,
    ngram_counts,
    context_counts,
    vocab,
    n=2,
    alpha=1.0,
    max_length=20
):
    """
    Generates text from an n-gram model until it sees </s>
    or reaches a maximum total length (max_length).

    Args:
      start_tokens (list): initial context to begin generation
      ngram_counts (Counter): trained n-gram counts
      context_counts (Counter): trained (n-1)-gram counts
      vocab (set): the model vocabulary
      n (int): n-gram order, 2 for bigram, 3 for trigram, etc.
      alpha (float): Laplace smoothing parameter
      max_length (int): maximum number of tokens to generate (including start_tokens)

    Returns:
      A list of tokens representing the generated sequence.
    """
    generated = start_tokens[:]
    
    while len(generated) < max_length:
        context = generated[-(n-1):]
        next_token_candidates = predict_next_token(context, ngram_counts, context_counts, vocab, n, alpha, top_k=1)
        
        if not next_token_candidates:
            break
        
        next_token = next_token_candidates[0][0]
        if next_token == '</s>':
            break
        
        generated.append(next_token)
    
    return generated

context = ["i", "love"]
generated_seq_unigram = generate_text_with_limit(
    start_tokens=context,
    ngram_counts=unigram_counts,
    context_counts=unigram_context_counts,
    vocab=vocab,
    n=1,
    alpha=1.0,
    max_length=128
)
print("Generated Sequence (Unigram):", generated_seq_unigram)

generated_seq_bigram = generate_text_with_limit(
    start_tokens=context,
    ngram_counts=bigram_counts,
    context_counts=bigram_context_counts,
    vocab=vocab,
    n=2,
    alpha=1.0,
    max_length=128
)
print("Generated Sequence (Bigram):", generated_seq_bigram)

generated_seq_trigram = generate_text_with_limit(
    start_tokens=context,
    ngram_counts=trigram_counts,
    context_counts=trigram_context_counts,
    vocab=vocab,
    n=3,
    alpha=1.0,
    max_length=128
)
print("Generated Sequence (Trigram):", generated_seq_trigram)

Generated Sequence (Unigram): ['i', 'love', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone', 'cyclone',

In [13]:
def calculate_perplexity(
    tokenized_sentences,
    ngram_counts,
    context_counts,
    vocab_size,
    n=2,
    alpha=1.0
):
    """
    Calculates the perplexity of an n-gram model (with Laplace smoothing)
    on a list of tokenized sentences.
    """
    log_prob_sum = 0.0
    total_words = 0

    for sentence in tokenized_sentences: 
        if isinstance(sentence, str):
            sentence = sentence.split()
            
        padded_sent = pad_sentence(sentence, n)
        total_words += len(sentence)
        
        for i in range(len(padded_sent) - n + 1):
            ngram = tuple(padded_sent[i:i + n])
            prob = laplace_probability(ngram, ngram_counts, context_counts, vocab_size, alpha)
            #log base e (natural logarithm)
            log_prob_sum += np.log(prob)
    
    #average negative log probability
    avg_log_prob = -1 * log_prob_sum / total_words
    return np.exp(avg_log_prob)

perplexity_unigram = calculate_perplexity(tokenized_test, unigram_counts, unigram_context_counts, len(vocab), n=1, alpha=1.0)
print(f"Perplexity (Unigram): {perplexity_unigram:.2f}")

perplexity_bigram = calculate_perplexity(tokenized_test, bigram_counts, bigram_context_counts, len(vocab), n=2, alpha=1.0)
print(f"Perplexity (Bigram): {perplexity_bigram:.2f}")

perplexity_trigram = calculate_perplexity(tokenized_test, trigram_counts, trigram_context_counts, len(vocab), n=3, alpha=1.0)
print(f"Perplexity (Trigram): {perplexity_trigram:.2f}")

Perplexity (Unigram): 1263.73
Perplexity (Bigram): 3586.43
Perplexity (Trigram): 39226.19
